# **Setup (keys, imports, DB path)**

In [6]:
# Minimal imports
import os, sqlite3, time
from datetime import datetime
import requests
import pandas as pd

# === RapidAPI creds ===
# Option A: set env var RAPIDAPI_KEY (recommended)
# Option B: paste your key below and leave the env empty
PASTE_KEY = "f5d6786ebdmsh4c0039968cab7abp1a67efjsnc575353b1079"  # put your key here if you don't use env vars

RAPIDAPI_KEY  = os.getenv("RAPIDAPI_KEY", "").strip() or PASTE_KEY.strip()
RAPIDAPI_HOST = "cricbuzz-cricket.p.rapidapi.com"
BASE_URL      = f"https://{RAPIDAPI_HOST}"

if not RAPIDAPI_KEY:
    raise ValueError("No API key found. Set env RAPIDAPI_KEY or paste in PASTE_KEY.")

HEADERS = {
    "X-RapidAPI-Key": RAPIDAPI_KEY,
    "X-RapidAPI-Host": RAPIDAPI_HOST,
}

# === SQLite DB path (local file) ===
DB_PATH = "cricket_test.db"


# **Tiny helper**

In [7]:
def get_json(url, params=None, retries=1, timeout=12):
    for attempt in range(retries + 1):
        r = requests.get(url, headers=HEADERS, params=params or {}, timeout=timeout)
        if r.status_code == 200:
            return r.json()
        time.sleep(0.6)
    r.raise_for_status()

# **Tables creation**

In [8]:
def init_db(db_path=DB_PATH):
    ddl = """
    CREATE TABLE IF NOT EXISTS matches (
        match_id INTEGER PRIMARY KEY,
        description TEXT,
        match_type TEXT,
        status TEXT,
        start_time TEXT,
        team1 TEXT,
        team2 TEXT,
        venue TEXT
    );
    """
    with sqlite3.connect(db_path) as conn:
        conn.execute(ddl)
        conn.commit()

init_db()
print("DB ready at:", os.path.abspath(DB_PATH))


DB ready at: /content/cricket_test.db


# **Fetch recent/live/upcoming matches from RapidAPI and normalize it**

In [9]:
KIND = "recent"  # change to "live" or "upcoming" if you want
data = get_json(f"{BASE_URL}/matches/v1/{KIND}")

def ts(ms):
    try:
        return datetime.utcfromtimestamp(int(ms)/1000).strftime("%Y-%m-%d %H:%M:%S")
    except Exception:
        return None

rows = []
for block in data.get("typeMatches", []):
    for wrapper in block.get("seriesMatches", []):
        series = wrapper.get("seriesAdWrapper") or {}
        for m in series.get("matches", []):
            mi = (m or {}).get("matchInfo", {}) or {}
            v  = mi.get("venueInfo", {}) or {}
            rows.append({
                "match_id"   : mi.get("matchId"),
                "description": mi.get("matchDesc"),
                "match_type" : mi.get("matchFormat"),
                "status"     : mi.get("stateTitle") or mi.get("status"),
                "start_time" : ts(mi.get("startDate")),
                "team1"      : (mi.get("team1") or {}).get("teamName"),
                "team2"      : (mi.get("team2") or {}).get("teamName"),
                "venue"      : v.get("ground"),
            })

df = pd.DataFrame(rows).dropna(subset=["match_id"]).drop_duplicates(subset=["match_id"])
print(f"Parsed {len(df)} matches from '{KIND}'.")
df.head(10)

Parsed 67 matches from 'recent'.


/tmp/ipython-input-254256000.py:6: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  return datetime.utcfromtimestamp(int(ms)/1000).strftime("%Y-%m-%d %H:%M:%S")


,match_id,description,match_type,status,start_time,team1,team2,venue
0,105786,1st ODI,ODI,RSA Won,2025-09-02 12:00:00,England,South Africa,Headingley
1,105794,3rd ODI,ODI,ENG Won,2025-09-07 10:00:00,England,South Africa,The Rose Bowl
2,105788,2nd ODI,ODI,RSA Won,2025-09-04 12:00:00,South Africa,England,Lord's
3,131086,6th Match,T20,AFG Won,2025-09-05 15:00:00,Afghanistan,United Arab Emirates,Sharjah Cricket Stadium
4,131092,Final,T20,PAK Won,2025-09-07 15:00:00,Pakistan,Afghanistan,Sharjah Cricket Stadium
5,131070,4th Match,T20,AFG Won,2025-09-02 15:00:00,Afghanistan,Pakistan,Sharjah Cricket Stadium
6,131075,5th Match,T20,PAK Won,2025-09-04 15:00:00,Pakistan,United Arab Emirates,Sharjah Cricket Stadium
7,128919,85th Match,ODI,Abandon,2025-09-06 14:30:00,Canada,Scotland,Maple Leaf North-West Ground
8,128908,84th Match,ODI,SCO Won,2025-09-04 14:30:00,Scotland,Namibia,Maple Leaf North-West Ground
9,128902,83rd Match,ODI,NAM Won,2025-09-02 14:30:00,Namibia,Canada,Maple Leaf North-West Ground


# **Upsert into SQlite and preview**

In [11]:
with sqlite3.connect(DB_PATH) as conn:
    if not df.empty:
        cols = list(df.columns)
        placeholders = ",".join(["?"] * len(cols))
        col_list = ",".join(cols)
        update_set = ",".join([f"{c}=excluded.{c}" for c in cols if c != "match_id"])
        sql = f"""
            INSERT INTO matches ({col_list})
            VALUES ({placeholders})
            ON CONFLICT(match_id) DO UPDATE SET {update_set};
        """
        conn.executemany(sql, [tuple(x) for x in df.itertuples(index=False, name=None)])
        conn.commit()

    preview = pd.read_sql_query("SELECT * FROM matches ORDER BY start_time DESC LIMIT 10;", conn)

print("Wrote rows:", len(df))
preview


Wrote rows: 67


,match_id,description,match_type,status,start_time,team1,team2,venue
0,131735,"1st Match, Group A",T20,FIJIW Won,2025-09-08 21:30:00,Cook Islands Women,Fiji Women,Albert Park 1
1,131741,"2nd Match, Group B",T20,PNGW Won,2025-09-08 21:30:00,Papua New Guinea Women,Japan Women,Albert Park 2
2,116774,25th Match,T20,SNP Won,2025-09-08 00:00:00,St Kitts and Nevis Patriots,Guyana Amazon Warriors,Providence Stadium
3,122093,2nd Match,T20,BRW Won,2025-09-07 19:00:00,Guyana Amazon Warriors Women,Barbados Royals Women,Providence Stadium
4,116765,24th Match,T20,BR Won,2025-09-07 15:00:00,Barbados Royals,Saint Lucia Kings,Kensington Oval
5,131092,Final,T20,PAK Won,2025-09-07 15:00:00,Pakistan,Afghanistan,Sharjah Cricket Stadium
6,133187,4th Match,T20,Abandon,2025-09-07 14:30:00,Isle of Man,Sweden,Cronkbourne Sports and Social Club Ground
7,130806,Final,T20,KBT Won,2025-09-07 13:15:00,Kochi Blue Tigers,Aries Kollam Sailors,Greenfield International Stadium
8,122973,3rd T20I,T20,SL Won,2025-09-07 11:30:00,Zimbabwe,Sri Lanka,Harare Sports Club
9,105794,3rd ODI,ODI,ENG Won,2025-09-07 10:00:00,England,South Africa,The Rose Bowl


# **Increasing the rows count**

In [13]:
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

# **Saving the test dataframe into csv**

In [14]:
csv_path = "sample_matches.csv"
df.to_csv(csv_path, index=False)
print("CSV saved at:", os.path.abspath(csv_path))

CSV saved at: /content/sample_matches.csv


In [15]:
pd.read_csv("sample_matches.csv")

,match_id,description,match_type,status,start_time,team1,team2,venue
0,105786,1st ODI,ODI,RSA Won,2025-09-02 12:00:00,England,South Africa,Headingley
1,105794,3rd ODI,ODI,ENG Won,2025-09-07 10:00:00,England,South Africa,The Rose Bowl
2,105788,2nd ODI,ODI,RSA Won,2025-09-04 12:00:00,South Africa,England,Lord's
3,131086,6th Match,T20,AFG Won,2025-09-05 15:00:00,Afghanistan,United Arab Emirates,Sharjah Cricket Stadium
4,131092,Final,T20,PAK Won,2025-09-07 15:00:00,Pakistan,Afghanistan,Sharjah Cricket Stadium
5,131070,4th Match,T20,AFG Won,2025-09-02 15:00:00,Afghanistan,Pakistan,Sharjah Cricket Stadium
6,131075,5th Match,T20,PAK Won,2025-09-04 15:00:00,Pakistan,United Arab Emirates,Sharjah Cricket Stadium
7,128919,85th Match,ODI,Abandon,2025-09-06 14:30:00,Canada,Scotland,Maple Leaf North-West Ground
8,128908,84th Match,ODI,SCO Won,2025-09-04 14:30:00,Scotland,Namibia,Maple Leaf North-West Ground
9,128902,83rd Match,ODI,NAM Won,2025-09-02 14:30:00,Namibia,Canada,Maple Leaf North-West Ground
